In [ ]:
from src import Color
from src.material.material.phong_material import PhongMaterial
from src import World
from src import Sphere, Plane
from src import PointLight
from src.scene.animation.animator import *
from src.render.post_process.post_process_config import PostProcessConfig
from src.render.loops.progress import PreviewConfig, ProgressDisplay
from src.render.render_config import RenderConfig
from src.render import LinearRayCaster
from src.render import MultiProcessRowRenderLoop

# Simple scene with a sphere and a plane and a light source
no skybox just to demonstrate basic functionality

In [ ]:
brown = Color.custom_rgb(150, 75, 0)
red = Color.custom_rgb(255, 0, 0)

glossy_red = PhongMaterial(name="glossy_red", base_color=red, spec_color=Color.custom_rgb(255, 255, 255), shininess=100, ior=1.0, reflectivity=0.3, transparency=0.0)
mate_blue = PhongMaterial(name="mate_blue", base_color=Color.custom_rgb(0, 0, 255), spec_color=Color.custom_rgb(128, 128, 128), shininess=10, ior=1.0, reflectivity=0.05, transparency=0.0)
glass = PhongMaterial(name="glass", base_color=Color.custom_rgb(200, 200, 255), spec_color=Color.custom_rgb(255, 255, 255), shininess=300, ior=1.5, reflectivity=0.05, transparency=0.9)
mirror = PhongMaterial(name="mirror", base_color=Color.custom_rgb(255, 255, 255), spec_color=Color.custom_rgb(255, 255, 255), shininess=500, ior=1.0, reflectivity=0.95, transparency=0.0)
glass_dark = PhongMaterial(name="glass_dark", base_color=Color.custom_rgb(150, 150, 200), spec_color=Color.custom_rgb(255, 255, 255), shininess=300, ior=1.5, reflectivity=0.05, transparency=0.9)
diamond = PhongMaterial(name="diamond", base_color=Color.custom_rgb(255, 255, 255), spec_color=Color.custom_rgb(255, 255, 255), shininess=500, ior=2.42, reflectivity=0.1, transparency=0.95)
oil = PhongMaterial(name="oil", base_color=Color.custom_rgb(255, 200, 0), spec_color=Color.custom_rgb(255, 255, 255), shininess=100, ior=1.2, reflectivity=0.0, transparency=0.95)
mate_purple = PhongMaterial(name="mate_purple", base_color=Color.custom_rgb(150, 0, 150), spec_color=Color.custom_rgb(128, 128, 128), shininess=10, ior=1.0, reflectivity=0.05, transparency=0.0)
gold = PhongMaterial(name="gold", base_color=Color.custom_rgb(255, 215, 0), spec_color=Color.custom_rgb(255, 223, 0), shininess=100, ior=0.47, reflectivity=0.8, transparency=0.0)

sphere = Sphere(center=Vertex(0, 0.2, -5), radius=1.0, material=glossy_red)
sphere2 = Sphere(center=Vertex(-1, 1.5, -3), radius=0.5, material=mate_blue)
glass_sphere = Sphere(center=Vertex(2, 0.5, -4), radius=0.5, material=glass)
mirror_sphere = Sphere(center=Vertex(0, 1.5, -6), radius=0.5, material=mirror)
diamond_sphere = Sphere(center=Vertex(-1.5, 1.0, -6), radius=0.6, material=diamond)
dark_glass_sphere = Sphere(center=Vertex(3, 1.0, -6), radius=0.5, material=glass_dark)
oil_sphere = Sphere(center=Vertex(0, -1.5, -4), radius=0.7, material=oil)
mate_purple_sphere = Sphere(center=Vertex(4, 0.3, -5), radius=0.3, material=mate_purple)
gold_sphere = Sphere(center=Vertex(-4, -1.2, -5), radius=0.3, material=gold)


world = World()
world.add(sphere, sphere2, glass_sphere, mirror_sphere, dark_glass_sphere, diamond_sphere, oil_sphere, mate_purple_sphere, gold_sphere)

point_light = PointLight(position=Vertex(5, 5, 0), intensity=2000.0, falloff=0.01)

camera = Camera(
    fov = 45,
    aspect_ratio = Resolution.R360p.aspect_ratio,
    origin = Vertex(0, 1.0, 2.0),
    direction = Vector(0, -0.2, -1),
)

scene = Scene(
    camera = camera,
    world = world,
    lights = [point_light],
    skybox_path = "./skybox/shanghai_4k.hdr"
)

scene.validate()

# Setup configurations for rendering

### Render a preview

In [ ]:
render_configuration = RenderConfig(
    resolution=Resolution.R144p,
    samples_per_pixel=1,
    max_depth=4,
)

preview_configuration = PreviewConfig(
    progress_display=ProgressDisplay.TQDM_IMAGE_PREVIEW,
)

ray_tracer = MultiProcessRowRenderLoop(
    scene=scene,
    render_config=render_configuration,
    preview_config=preview_configuration,
)
png = ray_tracer.render("animation_preview.png")

Render

In [ ]:
animation_setup = AnimationSetup(
    # camera move
    move_from = Vertex(0, 1.0, 2.0),
    move_to   = Vertex(-6, 0.1, 0.5),
    move_start_delay = 0.0,
    move_duration    = 2.0,

    # camera rotate yaw/pitch/roll
    rotate_axis      = Vector(-0.2, 1, 0.2),
    rotate_angle_deg = -55.0,
    rotate_start_delay = 0.0,
    rotate_duration    = 2.0,

    zoom_from       = 80.0,
    zoom_to         = 30.0,
    zoom_start_delay = 0.0,
    zoom_duration    = 2.0,
)


animator = Animator(
    animation_setup=animation_setup,
    animation_fps=12,
    animation_length_seconds=2.0,
    ray_tracer=ray_tracer,
)

animator.animate_to_mp4(
    ease=EaseType.EASE_IN_OUT
)